In [ ]:
### Initial design of drawing a coplanar waveguide resonator 


# automatic reloading of modules when they change

%load_ext autoreload
%autoreload 2


In [ ]:
# Import statements

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee

In [ ]:
# Instantiate QDesign class and initialise user interface

design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'
design.chips.main.size['size_z'] = '-500um'
design.chips.main.size['sample_holder_bottom'] = '500um'
design.chips.main.size['sample_holder_top'] = '2000um'

gui = MetalGUI(design)
# Set hfss as renderer for design
hfss = design.renderers.hfss

# Allow overwriting
design.overwrite_enabled = True


In [ ]:
# Add hanger of capacitievely coupled transmission lines
TQ1 = CoupledLineTee(design, 'TQ1', options=dict(pos_x='0mm',
                                                 pos_y='1.3mm',
                                                 coupling_length='100um',
                                                 coupling_space='3um'))
gui.rebuild()
gui.autoscale()

In [ ]:
# Create shorted pin and open pin for resonator
ops = dict(fillet='90um')
otg3 = OpenToGround(design, 'open_to_ground3', options=dict(pos_x='0mm', pos_y='-1.2mm',
orientation='270'))

route_options= Dict(total_length='8mm',
                   hfss_wire_bonds=False,
                   pin_inputs=Dict(start_pin=Dict(component='TQ1',
                                                 pin='second_end'),
                                  end_pin=Dict(component=otg3.name, pin='open')),
                   lead=Dict(start_straight='0.1mm'),
                   **ops)

# Create meandered resonator between shorted and opened pin
rt_meander = RouteMeander(design, 'readout', options=route_options)

gui.rebuild()
gui.autoscale()


In [ ]:
#Setup the launchpad location and orientation.
#launch_options1 = dict(pos_x='-500um', pos_y='0.5mm', orientation='0')

#lp = LaunchpadWirebondCoupled(design, 'P4_Q', options = launch_options1)

otg = OpenToGround(design, 'open_to_ground1', options=dict(pos_x='2mm', pos_y='1.3mm',
orientation='0'))



#Setup the OpenToGround location and orientation.
#launch_options2 = dict(pos_x='0.75mm',  pos_y='0.5mm', orientation='180')

#lp2 = LaunchpadWirebondCoupled(design, 'P5_Q', options = launch_options2)


otg2 = OpenToGround(design, 'open_to_ground', options=dict(pos_x='-2mm', pos_y='1.3mm',
orientation='180'))

#After the two QComponents are added to design, connect them with a RouteMeander.
meander_options_left = Dict(
        pin_inputs=Dict(
            start_pin=Dict(component='TQ1', pin='prime_start'),
            end_pin=Dict(component=otg2.name, pin='open')), )

meander_options_right = Dict(
        pin_inputs=Dict(
            start_pin=Dict(component='TQ1', pin='prime_end'),
            end_pin=Dict(component=otg.name, pin='open')), )


meander_left = RouteStraight(design, 'cpw_left',options=meander_options_left)
meander_right = RouteStraight(design, 'cpw_right',options=meander_options_right)

gui.rebuild()
gui.autoscale()

In [ ]:
# Screenshot the canvas only as a .png formatted file.
#gui.figure.savefig('shot.png')

#from IPython.display import Image, display
#_disp_ops = dict(width=500)
#display(Image('shot.png', **_disp_ops))

In [ ]:
hfss.open_ansys(path = 'C:\Program Files\AnsysEM\AnsysEM19.3\Win64')

In [ ]:
hfss.connect_ansys()

In [ ]:
hfss.activate_drivenmodal_design("S-parameter estimation")

In [ ]:
hfss.clean_active_design()

In [ ]:
hfss.render_design([],
                   [],
                  [('cpw_right', 'end', 50), ('cpw_left', 'end', 50)],
                  [],
                  [],
                  box_plus_buffer=False)

In [ ]:
design._chips['main']['size']['size_x'] = '3.012mm'
design._chips['main']['size']['size_y'] = '4mm'

In [ ]:
hfss.add_sweep(setup_name="Setup",
               name="Sweep",
               start_ghz=2.0,
               stop_ghz=6.0,
               count=16001,
               type="Interpolating")

In [ ]:

hfss.analyze_sweep('Sweep', 'Setup')

In [ ]:
hfss.plot_params(['S11', 'S21'])

In [ ]:
hfss.disconnect_ansys()